<h1 style="text-align:center;font-size:40px;">Tools 1 Final Project</h1>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<p style="text-align:center;">Anthony Molieri<br>
Leo Walker</p>
<br>
<br>
<br>
<br>

# Dataset and Motivation

The data used for these analyses were scraped from Indeed and Glassdoor. The data from Indeed contains job descriptions for open Data Scientist positions. The Glassdoor data includes employee generated ratings on benefits and perks for many of the companies present in the Indeed dataset. The datasets were then combined in order to give an idea of how employees rank the benefits at these companies.

*The Indeed data was scraped over ... pulling job descriptions for Data Scientists in 25 cities throughout the US. **Leo** Add Correct Details, here about the Indded data and scraper*

The Glassdoor scraper was designed to pull company names from the the job description dataset. It then looped over the list of employers, searched, and scraped ratings from the benefits page for companies that were found. This data was then added to it's own table in the same database. The scraper was designed to be run multiple times, only searching for benefits ratings that were not already present in the database. 

# Task Definition and Research Question
 
The purpose of this project was to help give a better idea of how employees at companies with open positions rate their benefits. This can help give insight into how desirable a company is as an employer.

# Literature Review

*Add -- I'm not really sure what to add here. Maybe something about turnover in data science? Companies not having great benefits?*

# Data Cleaning

In [37]:
# imports
import numpy as np
import pandas as pd
import sqlite3
import re

In [102]:
# Initialize both tables
cnct = sqlite3.connect('Data/jobs')

indeed_df = pd.read_sql_query('SELECT * FROM indeed_jobs;', cnct)
indeed_df.drop('est salary', axis=1, inplace=True)

glassdoor_df = pd.read_sql_query('SELECT companies, type, rating, count_of_ratings FROM ratings;', cnct)

In [103]:
indeed_df.columns

Index(['index', 'job_title', 'company_name', 'company_location', 'job_href',
       'job_desc', 'est_salary', 'retrieve_date'],
      dtype='object')

## Cleaning Indeed Data

In [148]:
# # Pull missing salary data from description
# salaryFix = indeed_df[['job_desc', 'est_salary']].copy()

# salaryFix['check1'] = salaryFix['job_desc'].str.contains("\$")
# salaryFix['check2'] = salaryFix['job_desc'].str.contains("salary")

# I'm not great at regex but let's try this
# Only look for salary data when it's missint in 'est_salary' 
# indeed_df['salary_extract'] = np.where(indeed_df['est_salary'] == "No Estimated Salary",indeed_df['job_desc'].str.extract("((\$)(([0-9])([0-9])([0-9])(,)([0-9])([0-9])([0-9])|([0-9])([0-9])(,)([0-9])([0-9])([0-9]))( to |-| - )(\$|)(([0-9])([0-9])([0-9])(,)([0-9])([0-9])([0-9])|([0-9])([0-9])(,)([0-9])([0-9])([0-9]))|(\$)([0-9][0-9][0-9],[0-9][0-9][0-9])|(\$)([0-9][0-9],[0-9][0-9][0-9]))")[0], np.NaN)

def salary_extract(row):
    if row['est_salary']!="No Estimated Salary":
        return row['est_salary']
    else:
        pattern = r"\$([0-9]{5,6}|[0-9]{2,3},[0-9]{3})"
        found = re.findall(pattern, row['job_desc'])
        if len(found)==1:
            return f'{found[0]}'
        elif len(found)>1:
            return f'{found[0]} - {found[1]}'
        else:
            return None
indeed_df['salary_extract'] = indeed_df.apply(salary_extract, axis=1)

In [105]:
def set_job_category(job_title):
    regex = re.compile('[^a-zA-Z]')
    clean_title = regex.sub('', job_title)
    clean_title = clean_title.lower()
    if all(x in clean_title for x in ["data", "scien"]):
        return "Data Scientist"
    elif all(x in clean_title for x in ["business", "analy"]):
        return "Business Intelligence Analyst"
    elif all(x in clean_title for x in ["bi", "analy"]):
        return "Business Intelligence Analyst"
    elif all(x in clean_title for x in ["data", "analy"]):
        return "Data Analyst"
    elif all(x in clean_title for x in ["machine", "learning"]):
        return "Machine Learning Engineer"
    elif all(x in clean_title for x in ["data", "engineer"]):
        return "Data Engineer"
    elif all(x in clean_title for x in ["software", "engin"]):
        return "Software Engineer"
    elif all(x in clean_title for x in ["architect"]):
        return "Data Architect"
    else:
        return "Other"
indeed_df['job_category'] = indeed_df.loc[:, 'job_title'].apply(set_job_category)

In [106]:
indeed_df.loc[:,'state'] = indeed_df.loc[:, 'company_location'].str.extract(r', ([A-Z]{2})')

In [149]:
# indeed_df[indeed_df['salary_extract']!=None]
for i in indeed_df.salary_extract:
    print(i)

None
None
None
None
None
None
$56,000 - $106,080 a year
None
None
None
$72,500 - $137,400 a year
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
$100,800 - $230,200 a year
None
None
None
None
None
$110,500 a year
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
$60,000 - $90,000 a year
$105,400 a year
$123,300 - $160,000 a year
78,718 - 129,885
84,300 - 125,999
None
90,000 - 150,000
None
$100,800 - $230,200 a year
None
86,200 - 176,600
$120,700 a year
$120,700 - $160,000 a year
90,000 - 140,000
$145,200 - $217,800 a year
None
None
None
None
None
None
None
None
None
None
64,800 - 116,000
None
None
None
None
None
None
None
None
84,300 - 125,999
None
None
None
None
None
None
None
None
None
None
None
None
$75,504 - $94,761 a year
None
None
None
None
None
None
None
None
None
None
None
None
None
None
20,000
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None

In [141]:
# indeed_df.info()
# indeed_df.loc[indeed_df['est_salary']=="No Estimated Salary"]
indeed_df.loc[236,'job_desc']

"You’ll enjoy the flexibility to telecommute* from anywhere within the U.S. as you take on some tough challenges. Combine two of the fastest - growing fields on the planet with a culture of performance, collaboration and opportunity and this is what you get. Leading edge technology in an industry that's improving the lives of millions. Here, innovation isn't about another gadget, it's about making Healthcare data available wherever and whenever people need it, safely and reliably. There's no room for error. Join us and start doing your life’s best work. SM This role performs exploratory analysis activities related to the Data Mining and CCM programs. The incumbent will review all types of medical claim payments against contracts, system configurations, client policies, and regulations. The incumbent will review ad-hoc reporting, new business implementations, client system migrations and client referrals to validate provider contractual terms in conjunction with client systems. The obje

In [89]:
def salary_low(row):
    if row =='No Estimated Salary':
        return None
    salary = str(row).split('-')
    pattern = r'([0-9]*)'
    if len(salary)>1:
        low = int(''.join(re.findall(pattern, salary[0])))
    else:
        low = int(''.join(re.findall(pattern, salary[0])))
    return low

def salary_high(row):
    if row =='No Estimated Salary':
        return None
    salary = str(row).split('-')
    pattern = r'([0-9]*)'
    if len(salary)>1:
        high = int(''.join(re.findall(pattern, salary[1])))
    else:
        low = high = int(''.join(re.findall(pattern, salary[0])))
    return high

indeed_df.loc[:,['salary_low']] = indeed_df.loc[:,'est_salary'].apply(salary_low)
indeed_df.loc[:,['salary_high']] = indeed_df.loc[:,'est_salary'].apply(salary_high)

In [17]:
def pay_rate(row):
    regex = re.compile('[.[0-9]]')
    clean_salary = regex.sub('', row)
    clean_salary = clean_salary.lower()
    if any(x in clean_salary for x in ["year"]):
        return "annual"
    elif any(x in clean_salary for x in ["month"]):
        return "month"
    elif any(x in clean_salary for x in ["week"]):
        return "week"
    elif any(x in clean_salary for x in ["day"]):
        return "day"
    elif any(x in clean_salary for x in ["hour"]):
        return "hour"
    else:
        return "Other"

indeed_df['pay_rate'] = indeed_df.loc[:, 'est_salary'].apply(pay_rate)

In [18]:
indeed_df[indeed_df['est_salary']!='No Estimated Salary'].sample(3)

,index,job_title,company_name,company_location,est salary,job_href,job_desc,est_salary,retrieve_date,salary_extract,job_category,state,salary_low,salary_high,pay_rate
14048,10,FP&A Analyst - Remote,Matrix Providers,"Remote in Orlando, FL",None,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Matrix Providers - Caring for America's milit...,"$75,000 - $80,000 a year",2022-04-04,NaN,Other,FL,75000.0,80000.0,annual
9699,10,Data Specialist,Richmond Public Schools,"Richmond, VA",None,https://www.indeed.com/rc/clk?jk=3ac3d86cfab2a...,Data Specialist Grade Level/Terms of Employme...,"$59,049 - $100,557 a year",2022-03-14,NaN,Other,VA,59049.0,100557.0,annual
16184,11,"Senior Data Engineer, i360 (Remote)",i360,"Remote in Chicago, IL 60606",None,https://www.indeed.com/rc/clk?jk=a1dda68fcea45...,Description This role is open to remote wor...,"$125,000 - $200,000 a year",2022-04-11,NaN,Data Engineer,IL,125000.0,200000.0,annual


In [23]:
rate_change= {'annual':1, 'month':12, 'week':52, 'day':260, 'hour':416}
rate_change['hour']

416

In [34]:
def calc_salary(row):
    # print(row)
    pay_rate=row['pay_rate']
    if pay_rate == 'Other':
        return None

    rate_change= {'annual':1, 'month':12, 'week':52, 'day':260, 'hour':2080}
    est_annual_salary = ((row['salary_low']+row['salary_high'])/2)*rate_change[pay_rate]
    return est_annual_salary
indeed_df['est_annual_salary'] = indeed_df.loc[:,['est_salary', 'salary_low','salary_high','pay_rate']].apply(calc_salary, axis=1)

In [36]:
indeed_df[indeed_df['est_salary']!='No Estimated Salary'].sample(4)

,index,job_title,company_name,company_location,est salary,job_href,job_desc,est_salary,retrieve_date,salary_extract,job_category,state,salary_low,salary_high,pay_rate,est_annual_salary
12308,2,Business Intelligence Analyst |ENTRY LEVEL|,PCS Globaltech,"Jacksonville, FL",None,https://www.indeed.com/company/PCS-Globaltech/...,Responsibilities: Developing and managing bu...,"$70,000 - $80,000 a year",2022-03-28,NaN,Business Intelligence Analyst,FL,70000.0,80000.0,annual,75000.0
12262,0,Business Intelligence Analyst *Entry Level*,PCS GLOBAL TECH,"Salt Lake City, UT+1 location",None,https://www.indeed.com/company/PCS-GLOBAL-TECH...,Responsibilities: Developing and managing bu...,"$58,245 - $129,979 a year",2022-03-28,NaN,Business Intelligence Analyst,UT,58245.0,129979.0,annual,94112.0
7918,1,Data Analyst,North Carolina Dept of Commerce,"Wake County, NC",None,https://www.indeed.com/rc/clk?jk=a18523202a19d...,"Recruitment Salary Range: $42,780 - $65,000. S...","$42,780 - $72,424 a year",2022-03-05,NaN,Data Analyst,NC,42780.0,72424.0,annual,57602.0
11533,1,Data Analyst,ATHENA Consulting,"Chicago, IL 60605 (The Loop area)",None,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,OVERVIEW OF ATHENA: ATHENA Consulting is an ...,$26 an hour,2022-03-28,NaN,Data Analyst,IL,26.0,26.0,hour,54080.0


In [24]:
indeed_df.salary_extract

'75000'

'$75,000 a year'

## Cleaning Glassdoor Data

# Visualizations